# 정령 시연 시나리오 (던전)  
## 1층 던전 입장  
- "인벤토리에 아이템이 뭐 있어?" (STT 사용 모습 보여줌)
- "xxxx 사용해" (장비무기 사용 상호작용)
- "던전의 밝기좀 켜줘" (던전 밝기 조절 on)

## 던전 플레이 도중
- "던전의 밝기좀 꺼줘" (던전 밝기 조절 off)

## 보스몬스터 조우
- "보스몬스터의 전투 패턴을 알려줘" 

In [1]:
import os, sys
from pathlib import Path


def find_src_folder():
    current = Path(os.getcwd()).resolve()
    for p in [current] + list(current.parents):
        src = p / "src"
        if src.exists():
            return src
    raise RuntimeError("src 폴더를 찾을 수 없습니다.")


src_path = find_src_folder()
sys.path.append(str(src_path))
from agents.fairy.dungeon.fairy_dungeon_agent import (
    graph_builder as dungeon_graph_builder,
)

from agents.fairy.util import add_human_message
from core.game_dto.z_muck_factory import MockFactory
from core.game_dto.StatData import StatData
from agents.fairy.fairy_state import DungeonPlayerState

from agents.fairy.interaction.fairy_interaction_agent import (
    graph_builder as interaction_builder,
)
from langchain.messages import HumanMessage
from agents.fairy.memory_messages import get_fairy_messages_dungeon
from core.common import get_inventory_items, get_skills


dungeon_graph = dungeon_graph_builder.compile()
interaction_graph = interaction_builder.compile()


async def invoke_graphs(
    question: str, target_monster_ids=[], weapon=MockFactory.create_weapon()
):
    inventory_ids = [3, 46, 67, 110]
    stats = StatData(
        strength=8,
        dexterity=1,
    )
    inventory = get_inventory_items(inventory_ids, stats)    
    skill_ids = [0, 1]
    skills = get_skills(skill_ids)

    player = DungeonPlayerState(
        playerId="TEST0",
        heroineId=1,
        currRoomId=1,
        difficulty=0,
        stats=stats,
        skills=skills,
        weapon=weapon,
        inventory=inventory,
    )

    memories = get_fairy_messages_dungeon(
        player_id=player.playerId, heroine_id=player.heroineId, limit=4
    )

    inputs = {
        "messages": memories + [add_human_message(content=question)],
        "dungenon_player": player,
        "target_monster_ids": target_monster_ids,
    }

    dungeon_talk = await dungeon_graph.ainvoke(inputs)

    # interaction_talk = interaction_graph.invoke(
    #     {
    #         "inventory": inventory_ids,
    #         "messages": memories + [HumanMessage(question)],
    #         "weapon": player.weapon,
    #         "stats": player.stats,
    #     }
    # )
    print("기억::", memories)
    print("질문::", dungeon_talk["messages"][-2])
    print("응답::", dungeon_talk["messages"][-1])
    # print(
    #     "상호 작용:",
    #     {
    #         "useItemId": interaction_talk["useItemId"],
    #         "roomLight": interaction_talk["roomLight"],
    #         "temp_use_item_id": interaction_talk["temp_use_item_id"],
    #         "is_item_use": interaction_talk["is_item_use"],
    #         "latency_create_temp_use_item_id": interaction_talk["latency_create_temp_use_item_id"]
    #     },
    # )


/Users/seobi/PythonProjects/ProjectML/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/seobi/PythonProjects/ProjectML/.venv/lib/python3.12/site-packages/pydantic/_internal/_config.py:383: UserWarning: Valid config keys have changed in V2:
* 'orm_mode' has been renamed to 'from_attributes'
  warnings.warn(message, UserWarning)
/Users/seobi/PythonProjects/ProjectML/.venv/lib/python3.12/site-packages/pydantic/main.py:250: UserWarning: WARNING! top_p is not default parameter.
                    top_p was transferred to model_kwargs.
                    Please confirm that top_p is what you intended.
  validated_self = self.__pydantic_validator__.validate_python(data, self_instance=self)
Fetching 30 files: 100%|██████████| 30/30 [00:00<00:00, 433893.52it/s]
You're using a XLMRobertaTokenizerFast

In [ ]:
# Q1. 인벤토리에 쌘무기 뭐 있어?
# 인벤토리에 레전드 숏소드, 레어 드래곤슬레이어, 레전드 바이킹의 망치 있어!' 
# 'useItemId': None, 'roomLight': 0, 'temp_use_item_id': None, 'is_item_use': False

await invoke_graphs("인벤토리에 쌘무기 뭐 있어?")

In [ ]:
# Q2. 그것 중에 가장 약한걸로 껴줘
# 인벤토리에 데미지가 가장 낮은 레어 드래곤슬레이어로 바꿨어! 이제 이걸로 싸울게.
# 'useItemId': 46, 'roomLight': 0, 'temp_use_item_id': 46, 'is_item_use': True

await invoke_graphs("그것 중에 가장 약한걸로 껴줘")

In [ ]:
# Q3. 무기 교체 해줘!
# 알겠어! 인벤토리에서 제일 강한 레전드 바이킹의 망치로 바꿔줄게.
# 'useItemId': 67, 'roomLight': 0, 'temp_use_item_id': 67, 'is_item_use': True
from core.game_dto.z_muck_factory import MockFactory
await invoke_graphs('무기 교체 해줘!', weapon= MockFactory.create_yakk_weapon())

In [ ]:
#4
await invoke_graphs('보스몬스터의 전투 패턴을 알려줘', target_monster_ids=[1001])

In [ ]:
#5
await invoke_graphs('던전의 밝기좀 켜줘')

In [ ]:
#6
await invoke_graphs("인벤토리에 뭐 있어?")

In [ ]:
#7
await invoke_graphs("중량의 반지 사용해")

In [ ]:
await invoke_graphs("고양이의 털 사용해")

In [ ]:
await invoke_graphs("아이템 사용해")

In [ ]:
await invoke_graphs("숏쇼드랑 드래곤 슬레이어 데미지 몇인데")

In [ ]:
await invoke_graphs("그럼 둘중 약한걸로 써줘", weapon= MockFactory.create_power_weapon())

In [ ]:
await invoke_graphs("고급 드워프의 망치 사용해")

In [ ]:
await invoke_graphs("레어 쌍검 사용해", weapon= MockFactory.create_power_weapon())

In [ ]:
await invoke_graphs("인벤토리에 아이템 뭐 있어?")

In [ ]:
await invoke_graphs("불 꺼줘")

In [ ]:
await invoke_graphs("스킬 사용법이 뭐야?")

In [ ]:
await invoke_graphs("저 보스몬스터는 뭐야?", target_monster_ids=[1000])

In [ ]:
await invoke_graphs("무기 스왑 해줘", target_monster_ids=[1000])

In [ ]:
await invoke_graphs("인벤토리에 악세사리 뭐있어?", target_monster_ids=[1000])

In [ ]:
await invoke_graphs("현재 방에 몬스터 얼마나 있어?", target_monster_ids=[1000])

In [ ]:
await invoke_graphs("저거 뭐야?", target_monster_ids=[1000])

In [ ]:
await invoke_graphs("사용키좀 알려줘", target_monster_ids=[1000])

In [ ]:
await invoke_graphs("이벤트 어떻게 꺠?", target_monster_ids=[1000])

In [ ]:
await invoke_graphs("내 인벤토리에 쓸만한 장비 2개만 말해봐")

In [ ]:
await invoke_graphs("둘중 쌘거 껴줘")

In [2]:
await invoke_graphs("내가 가진 스킬에 대해 알려줘")

총 질문 <Used Abilities>
['INTERACTION_HANDLER']
</Used Abilities>

<Heroine Info>
{
  "playerId": "TEST0",
  "heroineId": 1,
  "currRoomId": 1,
  "difficulty": 0,
  "stats": {
    "hp": 250,
    "moveSpeed": 1.0,
    "attackSpeed": 1.0,
    "cooldownReduction": 1.0,
    "strength": 8,
    "dexterity": 1,
    "intelligence": null,
    "critChance": 20.0,
    "skillDamageMultiplier": 1.0,
    "autoAttackMultiplier": 1.0
  },
  "skills": [
    {
      "skillId": 0,
      "skillName": "수리검",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "원거리, 빠른 공격속도"
    },
    {
      "skillId": 1,
      "skillName": "사자베기",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "근거리, 강한 한방, 넉백"
    }
  ],
  "weapon": {
    "weaponId": 22,
    "weaponType": 2,
    "weaponName": "레어 쌍검",
    "rarity": 2,
    "attackPower": 11,
    "staggerPower": 1,

In [3]:
await invoke_graphs("스킬 사용해")

총 질문 <Used Abilities>
['INTERACTION_HANDLER']
</Used Abilities>

<Heroine Info>
{
  "playerId": "TEST0",
  "heroineId": 1,
  "currRoomId": 1,
  "difficulty": 0,
  "stats": {
    "hp": 250,
    "moveSpeed": 1.0,
    "attackSpeed": 1.0,
    "cooldownReduction": 1.0,
    "strength": 8,
    "dexterity": 1,
    "intelligence": null,
    "critChance": 20.0,
    "skillDamageMultiplier": 1.0,
    "autoAttackMultiplier": 1.0
  },
  "skills": [
    {
      "skillId": 0,
      "skillName": "수리검",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "원거리, 빠른 공격속도"
    },
    {
      "skillId": 1,
      "skillName": "사자베기",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "근거리, 강한 한방, 넉백"
    }
  ],
  "weapon": {
    "weaponId": 22,
    "weaponType": 2,
    "weaponName": "레어 쌍검",
    "rarity": 2,
    "attackPower": 11,
    "staggerPower": 1,